In [46]:
import spacy
import random
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [47]:
df = pd.read_csv('location_articles.csv')

In [63]:
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.gold import biluo_tags_from_offsets

# Load the base English model
nlp = spacy.blank("en")

# Create and add the NER pipeline
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner, last=True)

# Define your labels (e.g., "LOCATION")
labels = ['LOC']

# Load and preprocess your training data
train_data = []
for body, entities in zip(df['body'], df['entities']):
    entities = eval(entities)
    train_data.append((body, {"entities": entities}))

# train_data = train_data[:50]

# Initialize the NER model with the labels
for label in labels:
    ner.add_label(label)

# Train the NER model
optimizer = nlp.begin_training()
for itn in range(10):  # Adjust the number of iterations as needed
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        nlp.update([text], [annotations], drop=0.5, losses=losses)
    print("Iteration:", itn+1, "Loss:", losses)

# Save the trained model
nlp.to_disk("trained_model")


C:\Users\Matthew Kenely\AppData\Roaming\Python\Python37\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
C:\Users\Matthew Kenely\AppData\Roaming\Python\Python37\site-packages\spacy\language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs


Iteration: 1 Loss: {'ner': 3904.038927143657}
Iteration: 2 Loss: {'ner': 232.49442892556527}
Iteration: 3 Loss: {'ner': 126.24215132050482}
Iteration: 4 Loss: {'ner': 69.88053068173063}
Iteration: 5 Loss: {'ner': 48.751502353647595}
Iteration: 6 Loss: {'ner': 19.55725848323732}
Iteration: 7 Loss: {'ner': 16.770940356570478}
Iteration: 8 Loss: {'ner': 15.175634675489546}
Iteration: 9 Loss: {'ner': 9.640996139780476}
Iteration: 10 Loss: {'ner': 9.907047247226737}
[]


In [73]:
df.loc[0:50]

,title,date,body,location,entities
0,"‘No one believes you’ on hospitals fiasco, Gre...","May 17, 2023 7:43 PM",Nationalist Party leader Bernard Grech insiste...,"['Attard', 'Attard', 'Attard', 'Attard', 'Atta...","[(4039, 4045, 'LOC'), (4169, 4175, 'LOC'), (44..."
1,PA starts demolishing illegal development in P...,"May 17, 2023 3:28 PM",The Planning Authority started removing illega...,['Pietà'],"[(76, 81, 'LOC')]"
2,Muscat revelations ‘textbook case of money lau...,"May 17, 2023 2:13 PM",Anticorruption NGO Repubblika has once again a...,['Valletta'],"[(274, 282, 'LOC')]"
3,Muscat received payments from Swiss firm that ...,"May 17, 2023 10:12 AM",Tista ' taqra bil Malti Disgraced former prime...,['Valletta'],"[(1482, 1490, 'LOC')]"
4,Occupy Justice demands reopening of Egrant inq...,"May 17, 2023 9:39 AM",Activists from Occupy Justice staged a protest...,['Floriana'],"[(774, 782, 'LOC')]"
5,"Repubblika demands arrest of Muscat, Mizzi to ...","May 16, 2023 3:07 PM",Tista ' taqra bil Malti Repubblika asked the m...,"['Valletta', 'Valletta']","[(1314, 1322, 'LOC'), (2153, 2161, 'LOC')]"
6,Muscat received payments from Swiss firm that ...,"May 17, 2023 10:12 AM",Tista ' taqra bil Malti Disgraced former prime...,['Valletta'],"[(1482, 1490, 'LOC')]"
7,"‘No one believes you’ on hospitals fiasco, Gre...","May 17, 2023 7:43 PM",Nationalist Party leader Bernard Grech insiste...,"['Attard', 'Attard', 'Attard', 'Attard', 'Atta...","[(4039, 4045, 'LOC'), (4169, 4175, 'LOC'), (44..."
8,New Magħtab ‘hill’ a temporary stockpile of ma...,"May 17, 2023 7:00 AM",A new ‘ hill ’ of construction waste that has ...,['Għargħur'],"[(1432, 1440, 'LOC')]"
9,PN to hold national protest on Wednesday,"May 16, 2023 6:55 PM",The Nationalist Party will be holding national...,['Attard'],"[(1751, 1757, 'LOC')]"


In [76]:
# Load the trained model
nlp = spacy.load("trained_model")

# Test the trained model on new article text
text = "I'd like to go to Valletta and meet my friend i paola"
doc = nlp(text)
entities = [(ent.text, ent.label_) for ent in doc.ents]
# predict location of text

print(entities)

[('Valletta', 'LOC')]


In [ ]:
test = df.loc[0]

In [ ]:
test.body

'Nationalist Party leader Bernard Grech insisted on Wednesday that no one believed what the Labour Party in government was saying on the hospitals concession granted to Vitals passed on to Steward and cancelled by the courts not least since the latest revelations saw the key protagonists openly contradict each other Grech and other PN MPs starting with Adrian Delia completely disregarded the agenda of Wednesday ’ s parliamentary sitting instead focusing solely on hospitals concession following revelations made by the Times and the Shift in collaboration with the Organised Crime and Corruption Reporting Project including that former PM Joseph Muscat received money from a firm that was previously called VGH Europe – an apparent reference to Vitals Global Healthcare A few minutes into Delia ’ s speech – the televised broadcast does not make clear whether he did so of his own accord or in response to informal complaints from the government benches – deputy speaker and PN MP David Agius rem

In [ ]:
eval(test.entities)

[(4039, 4045, 'LOC'),
 (4169, 4175, 'LOC'),
 (4406, 4412, 'LOC'),
 (4486, 4492, 'LOC'),
 (4833, 4839, 'LOC')]

In [ ]:
doc = nlp.make_doc(test.body)
tags = biluo_tags_from_offsets(doc, eval(test.entities))
print(tags)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',